# I.1 Integers


In this chapter we discuss the following:

1. Binary representation: Any real number can be represented in binary, that is,
by an infinite sequence of 0s and 1s (bits). We review  binary representation.
2. Unsigned integers:  We discuss how computers represent non-negative integers using only
$p$-bits, via [modular arithmetic](https://en.wikipedia.org/wiki/Modular_arithmetic).
3. Signed integers: we discuss how negative integers are handled using the [Two's-complement](https://en.wikipedia.org/wiki/Two's_complement) format.
4. As an advanced (non-examinable) topic we discuss `BigInt`, which uses variable bit length storage.

Before we begin its important to have a basic model of how a computer works. Our simplified model of a computer
will consist of a [Central Processing Unit (CPU)](https://en.wikipedia.org/wiki/Central_processing_unit)—the 
brains of the computer—and [Memory](https://en.wikipedia.org/wiki/Computer_data_storage#Primary_storage)—where 
data is stored. Inside the CPU there are [registers](https://en.wikipedia.org/wiki/Processor_register),
where data is temporarily stored after being loaded from memory, manipulated by the CPU, then stored back to memory. 

Memory is a sequence of bits: `1`s and `0`s, essentially "on/off" switches. These are grouped into bytes, which
consist of 8 bits. Each byte has a memory address: a unique number specifying its location in memory. The number
of possible addresses is limited by the processor: if a computer has a a $p$-bit CPU then each address is represented
by $p$ bits, for a total of $2^p$ addresses (on a modern 64-bit CPU this is $2^{64} ≈ 1.8 \times 10^{19}$ bytes).
Further, each register consists of exactly $p$-bits.

A CPU has the following possible operations:

1. load data from memory addresses (up to $p$-bits) to a register
2. store data from a register to memory addresses (up to $p$-bits)
3. Apply some basic functions ("+", "-", etc.) to the bits in one or two registers
and write the result to a register.

Mathematically, the important point is CPUs only act on $2^p$ possible sequences of bits at a time.
That is, essentially all functions $f$ implemented on a CPU are either of the form
$f : ℤ_{2^p} → ℤ_{2^p}$ or  $f : ℤ_{2^p} × ℤ_{2^p} → ℤ_{2^p}$,
where we use the following notation:

**Definition 1 ($ℤ_m$, signed integers)** Denote the
$$
ℤ_m := \{0 , 1 , …, m-1 \}
$$

The limitations this imposes on representing integers is substantial. 
If we have an implementation of $+$, which we shall denote $⊕ₘ$, how can we possibly represent
$m + 1$ in this implementation when the result is above the largest possible integer?

The solution that is used is straightforward: the CPU uses modular arithmetic. E.g., we have
$$
(m-1) ⊕ₘ 1 = m\ ({\rm mod}\ m) = 0.
$$
In this chapter we discuss the implications of this approach and how it works with negative numbers.


We will use Julia in these notes to explore what is happening as a computer does integer arithmetic.
We load an external package
which implements functions `printbits` (and `printlnbits`)
to print the bits (and with a newline) of numbers in colour:

In [1]:
using ColorBitstring

## 1.  Binary representation

Any integer can be presented in binary format, that is, a sequence of `0`s and `1`s.

**Definition 2 (binary format)**
For $B_0,\ldots,B_p \in \{0,1\}$ denote an integer in _binary format_ by:
$$
±(B_p\ldots B_1B_0)_2 := ±\sum_{k=0}^p B_k 2^k
$$



**Example 1 (integers in binary)**
A simple integer example is $5 = 2^2 + 2^0 = (101)_2$.
On the other hand, we write $-5 = -(101)_2$.
Another example is $258 = 2^8 + 2 = (1000000010)_2$.



## 2. Unsigned Integers


Computers represent integers by a finite number of $p$ bits,
with $2^p$ possible combinations of 0s and 1s. For _unsigned integers_ (non-negative integers) 
these bits dictate the first $p$ binary digits: $(B_{p-1}\ldots B_1B_0)_2$.
 
Integers on a computer follow [modular arithmetic](https://en.wikipedia.org/wiki/Modular_arithmetic):
Integers represented with $p$-bits on a computer actually 
represent elements of ${\mathbb Z}_{2^p}$ and integer arithmetic on a computer is 
equivalent to arithmetic modulo $2^p$. We denote modular arithmetic with $m = 2^p$ as follows:
$$
\begin{align*}
x ⊕_m y &:= (x+y) ({\rm mod}\ m) \\
x ⊖_m y &:= (x-y) ({\rm mod}\ m) \\
x ⊗_m y &:= (x-y) ({\rm mod}\ m)
\end{align*}
$$
When $m$ is implied by context we just write $⊕, ⊖, ⊗$.

**Example 2 (arithmetic with  8-bit unsigned integers)** 
If  arithmetic lies between $0$ and $m = 2^8 = 256$ works as expected. 
For example,
$$
\begin{align*}
17 ⊕_{256} 3 = 20 ({\rm mod}\ 256) = 20 \\
17 ⊖_{256} 3 = 14 ({\rm mod}\ 256) = 14
\end{align*}
$$
This can be seen in Julia:

In [2]:
x = UInt8(17)  # An 8-bit representation of the number 255, i.e. with bits 00010001
y = UInt8(3)   # An 8-bit representation of the number   1, i.e. with bits 00000011
printbits(x); println(" + "); printbits(y); println(" = ")
printlnbits(x + y) # + is automatically modular arithmetic
printbits(x); println(" - "); printbits(y); println(" = ")
printbits(x - y) # - is automatically modular arithmetic

00010001 + 
00000011 = 
00010100
00010001 - 
00000011 = 
00001110

**Example 3 (overflow with 8-bit unsigned integers)** If we go beyond the range
the result "wraps around". For example, with integers we have
$$
255 + 1 = (11111111)_2 + (00000001)_2 = (100000000)_2 = 256
$$
However, the result is impossible to store in just 8-bits! 
So as mentioned instead it treats the integers as elements of ${\mathbb Z}_{256}$:
$$
255 ⊕_{256} 1 = 255 + 1 \ ({\rm mod}\ 256) = (00000000)_2 \ ({\rm mod}\ 256) = 0 \ ({\rm mod}\ 256)
$$
We can see this in  code:

In [3]:
x = UInt8(255) # An 8-bit representation of the number 255, i.e. with bits 11111111
y = UInt8(1)   # An 8-bit representation of the number   1, i.e. with bits 00000001
printbits(x); println(" + "); printbits(y); println(" = ")
printbits(x + y) # + is automatically modular arithmetic

11111111 + 
00000001 = 
00000000

On the other hand, if we go below $0$ we wrap around from above:
$$
3 ⊖_{256} 5 = -2 ({\rm mod}\ 256) = 254 = (11111110)_2
$$

In [4]:
x = UInt8(3) # An 8-bit representation of the number   3, i.e. with bits 00000011
y = UInt8(5) # An 8-bit representation of the number   5, i.e. with bits 00000101
printbits(x); println(" - "); printbits(y); println(" = ")
printbits(x - y) # + is automatically modular arithmetic

00000011 - 
00000101 = 
11111110

**Example 4 (multiplication of 8-bit unsigned integers)** 
Multiplication works similarly: for example,
$$
254 ⊗_{256} 2 = 254 * 2 \ ({\rm mod}\ 256) = 252 \ ({\rm mod}\ 256) = (11111100)_2 \ ({\rm mod}\ 256)
$$
We can see this behaviour in code by printing the bits:

In [5]:
x = UInt8(254) # An 8-bit representation of the number 254, i.e. with bits 11111110
y = UInt8(2)   # An 8-bit representation of the number   2, i.e. with bits 00000010
printbits(x); println(" * "); printbits(y); println(" = ")
printbits(x * y)

11111110 * 
00000010 = 
11111100

### Hexadecimal and binary format

In Julia unsigned integers are displayed in hexadecimal
form: that is, in base-16.
Since there are only 10 standard digits (`0-9`) it uses 6 letters (`a–f`) to represent
11–16. For example,

In [6]:
UInt8(250)

0xfa

because `f` corresponds to 15 and `a` corresponds to 10, and we have
$$
15 * 16 + 10 = 250.
$$
The reason for this is that each hex-digit encodes 4 bits (since 4 bits have $2^4 = 16$ possible
values) and hence two hex-digits are encode 1 byte, and thus the digits correspond
exactly with how memory is divided into addresses.
We can create unsigned integers either by specifying their hex format:

In [7]:
0xfa

0xfa

Alternatively, we can specify their digits.
For example, we know $(f)_{16} = 15 = (1111)_2$ and $(a)_{16} = 10 = (1010)_2$ and hence
$250 = (fa)_{16} = (11111010)_2$ can be written as

In [8]:
0b11111010

0xfa

## 3. Signed integer

Signed integers use the [Two's complemement](https://epubs.siam.org/doi/abs/10.1137/1.9780898718072.ch3)
convention. The convention is if the first bit is 1 then the number is negative: the number $2^p - y$
is interpreted as $-y$.
Thus for $p = 8$ we are interpreting
$2^7$ through $2^8-1$ as negative numbers. More precisely:

**Definition 3 ($ℤ_{2^p}^s$, unsigned integers)**
$$
ℤ_{2^p}^s := \{-2^{p-1} ,…, -1 ,0,1, …, 2^{p-1}-1 \}
$$

**Definition 4 (Shifted mod)**
Define for $y = x\ ({\rm mod}\ 2^p)$
$$
x\ ({\rm mod}^{\rm s}\ 2^p) := \begin{cases} y & 0 ≤ y ≤ 2^{p-1}-1 \\
                             y - 2^p & 2^{p-1} ≤ y ≤ 2^p - 1
                             \end{cases}
$$

Note that if $R_p(x) = x ({\rm mod}^{\rm s}\ 2^p)$ then it can be viewed as a map
$R_p : ℤ → ℤ_{2^p}^s$ or a one-to-one map
$R_p : ℤ_{2^p} → ℤ_{2^p}^s$ whose inverse is $R_p^{-1}(x) = x \mod 2^p$. 


**Example 5 (converting bits to signed integers)** 
What 8-bit integer has the bits `01001001`? Because the first bit is 0 we know the result is positive.
Adding the corresponding decimal places we get:

In [9]:
2^0 + 2^3 + 2^6

73

What 8-bit (signed) integer has the bits `11001001`? Because the first bit is `1` we know it's a negative 
number, hence we need to sum the bits but then subtract `2^p`:

In [10]:
2^0 + 2^3 + 2^6 + 2^7 - 2^8

-55

We can check the results using `printbits`:

In [11]:
printlnbits(Int8(73)) # Int8 is an 8-bit representation of the signed integer 73
printbits(-Int8(55))

01001001
11001001

Arithmetic works precisely
the same for signed and unsigned integers, e.g. we have
$$
x ⊕_{2^p}^s y := x + y ({\rm mod}^{\rm s}\ 2^p)
$$

**Example 6 (addition of 8-bit integers)**
Consider `(-1) + 1` in 8-bit arithmetic. The number $-1$ has the same bits as
$2^8 - 1 = 255$. Thus this is equivalent to the previous question and we get the correct
result of `0`. In other words:
$$
-1 ⊕_{256} 1 = -1 + 1 \ ({\rm mod}\ 2^p) = 2^p-1  + 1 \ ({\rm mod}\ 2^p) = 2^p \ ({\rm mod}\ 2^p) = 0 \ ({\rm mod}\ 2^p)
$$


**Example 7 (multiplication of 8-bit integers)**
Consider `(-2) * 2`. $-2$ has the same bits as $2^{256} - 2 = 254$ and $-4$ has the
same bits as $2^{256}-4 = 252$, and hence from the previous example we get the correct result of `-4`.
In other words:
$$
(-2) ⊗_{2^p}^s 2 = (-2) * 2 \ ({\rm mod}^{\rm s}\ 2^p) = (2^p-2) * 2 \ ({\rm mod}^{\rm s}\ 2^p) = 2^{p+1}-4 \ ({\rm mod}^{\rm s}\ 2^p) = -4
$$





**Example 8 (overflow)** We can find the largest and smallest instances of a type using `typemax` and `typemin`:

In [12]:
printlnbits(typemax(Int8)) # 2^7-1 = 127
printbits(typemin(Int8)) # -2^7 = -128

01111111
10000000

As explained, due to modular arithmetic, when we add `1` to the largest 8-bit integer we get the smallest:

In [13]:
typemax(Int8) + Int8(1) # returns typemin(Int8)

-128

This behaviour is often not desired and is known as _overflow_, and one must be wary
of using integers close to their largest value.



### Division

 In addition to `+`, `-`, and `*` we have integer division `÷`, which rounds towards zero:

In [14]:
5 ÷ 2 # equivalent to div(5,2)

2

Standard division `/` (or `\` for division on the right) creates a floating-point number,
which will be discussed in the next chapter:

In [15]:
5 / 2 # alternatively 2 \ 5

2.5

We can also create rational numbers using `//`:

In [16]:
(1//2) + (3//4)

5//4

Rational arithmetic often leads to overflow so it
is often best to combine `big` with rationals:

In [17]:
big(102324)//132413023 + 23434545//4243061 + 23434545//42430534435

26339037835007648477541540//4767804878707544364596461

## 4. Variable bit representation (non-examinable)

An alternative representation for integers uses a variable number of bits,
with the advantage of avoiding overflow but with the disadvantage of a substantial
speed penalty. In Julia these are `BigInt`s, which we can create by calling `big` on an
integer:

In [18]:
x = typemax(Int64) + big(1) # Too big to be an `Int64`

9223372036854775808

Note in this case addition automatically promotes an `Int64` to a `BigInt`.
We can create very large numbers using `BigInt`:

In [19]:
x^100

3082994025277634745700106821545665721371798533305697458855342277921093731984476404705966539412410898240561729912372038501228893141921080152404642393776599077294434061519905424124601394226943601430916434383714716724720227331596950613701661034548948388721097667275438763758128508403297199458260277707301202460980093818414167080563342761482395862435185093942443540722363151770022221783243959592531336062998494209914752408019060720805124534382646051093613814848646062038662423487504326044361203708430489305864234333801401547140023376295718383390360728662900230671437151716615826286842267917560749586018165739492101920429719261285640125596833063891562865262157026023955919873792846823095854484520920509345944712871675691790827690907778485058829248588945681685288179787963931181062068092463984296225973082494056307958089189726701678735576365394146232076917088075949053636690459581128773097212746967276496496010810878000638239143750075543163240049874489986642327436441234458040254480825038220479904594615300

Note the number of bits is not fixed, the larger the number, the more bits required 
to represent it, so while overflow is impossible, it is possible to run out of memory if a number is
astronomically large: go ahead and try `x^x` (at your own risk).